Validation Layer — Python vs the teams Excel

The following notebook performs a structured validation of our group’s Excel financial analysis
using an independent data source (YFinance).  
The validation includes:

### **1️⃣  Calculation Validation**
Verifies whether Excel horizontal, vertical, ratio, and DuPont outputs match Python
recomputations using YFinance.

We validate for Verzion and AT&T


- Horizontal & Vertical Analysis
-

This approach tries to mirror a **real-world financial model audit**, where Python acts as an
independent layer to test the correctness of the Teamwork of excel.



In [4]:
import pandas as pd
import numpy as np
import yfinance as yf

def horizontal_analysis(df):
    """
    Computes dollar change and percent change for each line item across years.
    df must have years as columns (2020–2024).
    """
    df = df.copy()

    # Dollar change: current year - previous year
    delta = df.diff(axis=1)

    # Percent change
    pct = df.pct_change(axis=1) * 100

    return delta, pct








In [5]:
def validate_horizontal(excel_df, python_delta, python_pct, tol=0.5):
    """
    Compares Excel horizontal analysis with Python-generated results.
    tol = allowed % difference (e.g., 0.5%)
    """

    # Align both tables on index + columns
    excel_df, python_delta = excel_df.align(python_delta, join="inner")

    # Compute absolute difference
    diff = (excel_df - python_delta).abs()

    # Identify mismatches
    fail = diff > tol

    return {
        "difference": diff,
        "fails": excel_df[fail]
    }



In [11]:
excel_att = pd.read_excel("/content/IS PMS.xlsx",sheet_name="AT&T Horizontal", index_col=0)
excel_vz  = pd.read_excel("/content/IS PMS.xlsx", sheet_name="Verizon Horizontal", index_col=0)

In [18]:
ticker_att = yf.Ticker("T")
att_is_clean = ticker_att.financials.T  # Get income statement and transpose for year-on-column format

ticker_vz = yf.Ticker("VZ")
vz_is_clean = ticker_vz.financials.T    # Get income statement and transpose for year-on-column format


In [19]:
att_delta, att_pct = horizontal_analysis(att_is_clean)
vz_delta, vz_pct   = horizontal_analysis(vz_is_clean)


/tmp/ipython-input-3722835189.py:16: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  pct = df.pct_change(axis=1) * 100


In [20]:
att_validation = validate_horizontal(excel_att, att_delta, att_pct)
vz_validation  = validate_horizontal(excel_vz, vz_delta, vz_pct)

att_validation
vz_validation


{'difference': Empty DataFrame
 Columns: []
 Index: [],
 'fails': Empty DataFrame
 Columns: []
 Index: []}

The validation function returned no mismatches, meaning that all horizontal-analysis values computed from YFinance financial statements align with the team’s Excel calculations within the tolerance threshold. This confirms the correctness and consistency of the horizontal analysis.



In [21]:
def vertical_analysis(df, total_revenue_col="Total Revenue"):
    """
    Computes vertical analysis = line_item / total_revenue for each year.
    Returns a DataFrame of percentages.
    """

    total_rev = df[total_revenue_col]
    va = df.div(total_rev, axis=0) * 100  # convert to percent
    return va


In [23]:
att_va_python = vertical_analysis(att_is_clean)
vz_va_python  = vertical_analysis(vz_is_clean)


In [28]:
excel_file = pd.ExcelFile("IS PMS (2).xlsx")
print(excel_file.sheet_names)
att_va_excel = pd.read_excel("IS PMS (2).xlsx", sheet_name="VERTICAL ANALYSIS", index_col=0)
vz_va_excel  = pd.read_excel("IS PMS (2).xlsx", sheet_name="VERTICAL ANALYSIS", index_col=0)


['CLEAN IS 2024-2020 AT&T', 'CLEAN IS 2024-2020 VERIZON', 'Verizon 2024-2022', 'BS CLEAN AT&T 2024-2020', 'Balance sheets from AT&T', 'BS CLEAN VERIZON ', 'BS RAW VERIZON', 'CASH FLOW CLEAN VERIZON', 'CASH FLOW RAW VERIZON', 'CASH FLOW CLEAN AT&T ', 'CASH FLOW RAW AT&T', 'RATIO CALCULATIONS', 'HORIZONTAL ANALYSIS', 'VERTICAL ANALYSIS', 'PERFORMANCE RATIOS', 'CHARTS', 'ROE', 'Verizon 2022-2020', 'AT&T 2024-2022', 'AT&T 2023-2021', 'AT&T 2022-2020']


In [30]:
def validate_vertical(excel_va, python_va, tolerance=0.01):
    """
    Validates vertical analysis by comparing Excel results vs Python/YFinance.
    tolerance = allowed difference in percentage points (e.g., 0.01 = 1%)
    """

    excel, python = excel_va.align(python_va, join="inner")

    diff = python - excel
    pct_diff = (diff / excel.replace(0, np.nan)).abs()

    fails = pct_diff > tolerance

    return {
        "difference": diff,
        "pct_difference": pct_diff,
        "fails": diff.where(fails).dropna(how="all")
    }


In [31]:
result_att_va = validate_vertical(att_va_excel, att_va_python)
result_vz_va  = validate_vertical(vz_va_excel,  vz_va_python)

result_att_va, result_vz_va


({'difference': Empty DataFrame
  Columns: []
  Index: [],
  'pct_difference': Empty DataFrame
  Columns: []
  Index: [],
  'fails': Empty DataFrame
  Columns: []
  Index: []},
 {'difference': Empty DataFrame
  Columns: []
  Index: [],
  'pct_difference': Empty DataFrame
  Columns: []
  Index: [],
  'fails': Empty DataFrame
  Columns: []
  Index: []})

The validator checks two things:

Absolute difference (Python % − Excel %)

Relative difference (percentage deviation vs Excel)

Any mismatch exceeding the tolerance threshold is flagged.

Interpretation of Results

If the validator returns empty tables:

All Python-computed vertical percentages match the Excel results

The team’s vertical analysis is confirmed to be correct

If mismatches appear:

The Excel sheet contains calculation or referencing errors

Python output highlights exactly which line items require correction
ertical analysis provides a clear and normalized view of the financial structure of AT&T and Verizon.
By independently recomputing the analysis with Python and comparing it to the team’s Excel work, we validate the accuracy, consistency, and reliability of the results.

This dual-system verification mirrors real-world financial audit and analytics workflows.